In [1]:

import os
import sys
sys.path.append("..")
import numpy as np

import torch
from torch.utils.data import DataLoader,Subset
import torchvision.transforms as transforms

import torch.nn as nn
from dataset.defect_dataset import DefectDataset
from model.models import UNet
from training.train_model import training_model
from torch.nn import BCEWithLogitsLoss
from torchmetrics.classification import BinaryJaccardIndex
results_path = "test_results"
unet_config = {
        "encChannels": [1,8,16,32,64],
        "decChannels": [64,32,16,8],
        "retainDim": True,
        "input_image_height":256,
        "input_image_width":128
    }

data_path = "D:\\APPLY_JOB_2023\\Data\\KolektorSDD-boxes"


image_size = [256,128]

batch_size = 16
num_workers:int = 0
learningrate: int = 1e-3
weight_decay: float = 1e-5
nb_epochs: int = 10

c:\Users\DuyDat\anaconda3\envs\pythonENV\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load or download source dataset
tr_forms = transforms.ToTensor()
source_dl = DefectDataset(data_dir=data_path,img_size=image_size)

#source_dl = SourceData(**data_config,train_transformer(**data_config))

# Split source into training, validation and test set
trainingset = Subset(source_dl,indices=np.arange(int(len(source_dl) * (4 / 5))))
validationset = Subset(source_dl, indices=np.arange(int(len(source_dl) * (4 / 5)), len(source_dl)))






print(f"[INFO] found {len(trainingset)} examples in the SOURCE training set...")

print(f"[INFO] found {len(validationset)} examples in the source validationset set...")

# Create datasets and dataloaders 
trainloader = DataLoader(trainingset, batch_size, shuffle=False, num_workers=num_workers)
valloader = DataLoader(validationset, batch_size, shuffle=False, num_workers=num_workers)



# Create Network
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"[INFO] device is {device}")
model_Seg = UNet(**unet_config).to(device)



# Get loss function
criterion_seg = nn.BCELoss()
criterion_domain = BCEWithLogitsLoss()
jaccard = BinaryJaccardIndex(threshold=0.5)
# Get adam optimizer
optimizer_seg = torch.optim.Adam(
        model_Seg.parameters(),
        lr=learningrate,
        weight_decay=weight_decay, betas=(0.5, 0.99)
    )

[INFO] found 319 examples in the SOURCE training set...
[INFO] found 80 examples in the source validationset set...
[INFO] device is cuda:0


In [3]:
for batch,(X,y) in enumerate(trainloader):
    print("batch",batch)
    print(X.shape)
    print(y.shape)
    break
    

d:\APPLY_JOB_2023\Defect_Detection\Surface_DD\training\..\dataset\defect_dataset.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  return (torch.tensor([img_array]),torch.tensor([label_array]))


batch 0
torch.Size([16, 1, 256, 128])
torch.Size([16, 1, 256, 128])


In [4]:
from torch import tensor
from torchmetrics import JaccardIndex
jaccard_index = JaccardIndex(task='binary',num_classes=2)
target = tensor([[0, 1, 0], [1, 0, 1]])
preds = tensor([[0.11, 0.22, 0.84], [0.73, 0.33, 0.92]])
jaccard_index(preds, target)


tensor(0.5000)

In [7]:
epoch  = 0
X,y = X.float().to(device),y.to(device)

optimizer_seg.zero_grad()

domain_label_src = torch.ones((len(X),1)).to(device)


# Segmentation loss:
pred = model_Seg(X)
# seg_loss = criterion_seg(pred,y)
# IOU score:
# iou_score = jaccard_index(pred,y)
pred_class = pred.argmax(dim=1)
# iou_score = jaccard(src_pred_class,src_y)
print(pred_class.shape)
print(pred.shape)

torch.Size([16, 256, 128])
torch.Size([16, 1, 256, 128])


In [9]:
pred[1,0,:10,:10]

tensor([[0.1244, 0.1244, 0.1244, 0.1244, 0.1229, 0.1229, 0.1229, 0.1229, 0.1226,
         0.1226],
        [0.1244, 0.1244, 0.1244, 0.1244, 0.1229, 0.1229, 0.1229, 0.1229, 0.1226,
         0.1226],
        [0.1234, 0.1234, 0.1234, 0.1234, 0.1231, 0.1231, 0.1231, 0.1231, 0.1229,
         0.1229],
        [0.1234, 0.1234, 0.1234, 0.1234, 0.1231, 0.1231, 0.1231, 0.1231, 0.1229,
         0.1229],
        [0.1255, 0.1255, 0.1255, 0.1255, 0.1244, 0.1244, 0.1244, 0.1244, 0.1235,
         0.1235],
        [0.1233, 0.1233, 0.1233, 0.1233, 0.1254, 0.1254, 0.1254, 0.1254, 0.1233,
         0.1233],
        [0.1233, 0.1233, 0.1233, 0.1233, 0.1254, 0.1254, 0.1254, 0.1254, 0.1233,
         0.1233],
        [0.1247, 0.1247, 0.1247, 0.1247, 0.1256, 0.1256, 0.1256, 0.1256, 0.1255,
         0.1255],
        [0.1242, 0.1242, 0.1242, 0.1242, 0.1265, 0.1265, 0.1265, 0.1265, 0.1240,
         0.1240],
        [0.1242, 0.1242, 0.1242, 0.1242, 0.1265, 0.1265, 0.1265, 0.1265, 0.1240,
         0.1240]], device='c

In [10]:
a = torch.Tensor([[[0.2,0.9],[0.1,0.2]]])

In [15]:
a.argmax(dim=0).shape

torch.Size([2, 2])

In [8]:
pred_class[1,:10,:10]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [6]:
print(seg_loss.)

SyntaxError: invalid syntax (1367090484.py, line 1)

In [ ]:
from tqdm import tqdm
dataloader = source_valloader
is_bar_progress = False
jaccard = JaccardIndex(task ="multiclass",num_classes=nb_classes).to(device=device)
if is_bar_progress:
    update_progress_bar = tqdm(total=len(dataloader),desc=f"Evaluating: scoring: {np.nan:7.5f};iou scoring: {np.nan:7.5f}",position=0)
model_Seg.to(device)
model_Seg.eval()
mean_score = 0
count_batch = 0
mean_iou_score = 0
with torch.no_grad():
    for _,(X,y,y_lables) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        y_pred,_ = model_Seg(X)
        iou_score = jaccard(y_pred,y)
        mean_iou_score =(mean_iou_score*count_batch + iou_score)/(count_batch+1)
        mean_score = (mean_score*count_batch + criterion_seg(y_pred,y))/(count_batch+1)
        count_batch += 1
        if is_bar_progress:
            update_progress_bar.set_description(f"Evaluating: scoring: {mean_score:7.5f}; iou scoring: {mean_iou_score:7.5f}", refresh=True)
            update_progress_bar.update()
    if is_bar_progress:
        update_progress_bar.close()
model_Seg.train()
print( mean_score,mean_iou_score)